# TIF360 Project

# Data Pre-processing

Main source: https://www.kaggle.com/code/rmonge/predicting-molecule-properties-based-on-its-smiles/notebook

### Import packages

UPDATED ON 15/05-2023  11:20

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import math
import atomInSmiles
from torch.utils.data import DataLoader

import sys
sys.path.append('../')
from code_graphs.utility_functions import get_num_parameters , get_data_split_indices, scale_targets
from classes import create_encoded_vector, PositionalEncoding



import random


In [2]:

# check if cuda is available
print('cuda available:', torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', "cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('cuda version:', torch.version.cuda)
    print('gpu:', torch.cuda.get_device_name(0))



cuda available: True
device: cuda
cuda version: 11.7
gpu: NVIDIA GeForce GTX 970


In [3]:


df = pd.read_csv("../data/smiles_and_targets.csv")
print(np.shape(df))
mol_fingerprint = np.load("../data/mol_morgan_fingerprints.npy")


properties_names = ['A', 'B', 'C', 'mu', 'alfa', 'homo', 'lumo', 'gap', 'R²', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']
x_smiles = df.smiles.values
targetTensor = df.loc[:, properties_names].values # shape = (n_samples, n_properties)


(132820, 21)


In [4]:
# tokenize all smiles
#import atomInSmiles

tokenList = []
for smile in x_smiles:
    tokenList.append(atomInSmiles.encode(smile).split())
#tokenList is target

# find longest word (max tokens from one smile+1)
maxTokenLength = 0
for token in tokenList:
    if len(token)>maxTokenLength:
        maxTokenLength = len(token)
print('Longest word (max amount of tokens):', maxTokenLength)


# Give each token a index in a dictionary
tokenDict = {}
count = 1

dictList = []
for itokens in tokenList:
    for itoke in itokens:
        #print(itoke)
        if tokenDict.get(itoke) == None:
            tokenDict[itoke] = count
        
            # current = [itoke, count]
            # dictList.append(current)
            count+=1

Longest word (max amount of tokens): 22


In [5]:
encodedTokens = []
for token in tokenList:
    encodedTokens.append(create_encoded_vector(token,tokenDict,maxTokenLength))
    
encodedTokens = np.array(encodedTokens)


In [6]:

class TransformerLayer(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_channels,num_heads):
        super().__init__()
        self.Attention = torch.nn.MultiheadAttention(embedding_dim,num_heads=num_heads,dropout=0.15)
        self.Norm1 = torch.nn.LayerNorm(embedding_dim)
        self.Dense1 = torch.nn.Linear(embedding_dim,hidden_channels)
        self.relu = torch.nn.ReLU()
        self.Dense2 = torch.nn.Linear(hidden_channels,embedding_dim)
        

        self.Norm2 = torch.nn.LayerNorm(embedding_dim)
        

    def forward(self, x):
        addNormX = x
        #print(x.shape)
        x, _ = self.Attention(x,x,x)
        #print('attention output',x.shape)
        x = self.Norm1(x + addNormX)
        #print('norm + input',x.shape)
        addNormX = x
        x = self.Dense1(x)
        x = self.relu(x)
        #print('first dense output',x.shape)
        x = self.Dense2(x)
        #print('second dense output',x.shape)
        x = self.Norm2(x + addNormX)
        #print(x.shape)

 
        return x

In [7]:
class TransformerNetwork(torch.nn.Module):
    def __init__(self,hidden_channels,output_dim, vocab_size, embedding_dim, num_heads,maxTokenLength, nFingerprints):
        super().__init__()
        # Embedd and add pos encoding to input
        self.EmbeddingLayer = torch.nn.Embedding(num_embeddings=vocab_size,embedding_dim = embedding_dim , max_norm=True)
        self.PositionalEncoding = PositionalEncoding(embedding_dim, maxTokenLength, dropout = 0.15)

        self.TransEnc1 = TransformerLayer(embedding_dim,hidden_channels,num_heads)
        self.TransEnc2 = TransformerLayer(embedding_dim,hidden_channels,num_heads)
        self.TransEnc3 = TransformerLayer(embedding_dim,hidden_channels,num_heads)
        self.Pooling = torch.nn.AvgPool1d(kernel_size= 22)

        self.DenseOut1 = torch.nn.Linear(embedding_dim+nFingerprints,hidden_channels)
        self.DenseOut2 = torch.nn.Linear(hidden_channels,hidden_channels)
        self.DenseOut3 = torch.nn.Linear(hidden_channels,output_dim)
        self.relu = torch.nn.ReLU()


    def forward(self,x,fingerprints):
        x = self.EmbeddingLayer(x)
        x = self.PositionalEncoding(x)
        x = self.TransEnc1(x)
        x = self.TransEnc2(x)
        x = self.TransEnc3(x)
        x = self.Pooling(x.permute((0,2,1))).permute((0,2,1))
        x = torch.squeeze(x,axis=1)
        x = torch.cat((x,fingerprints),1)
        #x = x[:,-1,:]


        x = self.DenseOut1(x)
        x = self.relu(x)
        x = self.DenseOut2(x)
        x = self.relu(x)
        x = self.DenseOut3(x)
        
        return x

In [8]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler
batch_size = 64
## Create iterable dataset class:

class datasetObject(Dataset):
    def __init__(self,data,targets,fingerprints):
        self.data = data
        self.fingerprints = fingerprints
        self.targets = targets
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        sample = self.data[index]
        target = self.targets[index]
        descriptor = self.fingerprints[index]
        return (sample), (target), (descriptor)

trainSplit, validationSplit, testSplit = get_data_split_indices(len(encodedTokens),0.1,0.1)

# Training
encodedTrainData = torch.tensor(encodedTokens[trainSplit], dtype=torch.long, device=device)
trainTargets = targetTensor[trainSplit]

# Validation
encodedValidationData = torch.tensor(encodedTokens[validationSplit], dtype=torch.long, device=device)
validationTargets = targetTensor[validationSplit]

# Test
encodedTestData = torch.tensor(encodedTokens[testSplit], dtype=torch.long, device=device)
testTargets = targetTensor[testSplit]

# fingerprints
trainfingerprints = mol_fingerprint[trainSplit]
valfingerprints = mol_fingerprint[validationSplit]
testfingerprints = mol_fingerprint[testSplit]


trainTargets, validationTargets, testTargets, scalerTargets = scale_targets(trainTargets,validationTargets,testTargets)


# Normalize fingerprints and fingerprints using minmax scaler
minmax_scaler = MinMaxScaler()
trainfingerprints = minmax_scaler.fit_transform(trainfingerprints)
valfingerprints = minmax_scaler.transform(valfingerprints)
testfingerprints = minmax_scaler.transform(testfingerprints)


# Make into tensors
#print(np.dtype(trainTargets[0,0]))
trainTargets = torch.tensor(trainTargets,dtype=torch.float, device=device)
validationTargets = torch.tensor(validationTargets,dtype=torch.float,device=device)
testTargets = torch.tensor(testTargets,dtype=torch.float,device=device)

# Decriptors to tensor
trainfingerprints = torch.from_numpy(trainfingerprints).float().to(device)
valfingerprints = torch.from_numpy(valfingerprints).float().to(device)
testfingerprints = torch.from_numpy(testfingerprints).float().to(device)

trainingData = DataLoader(datasetObject(encodedTrainData,trainTargets,trainfingerprints),batch_size)
testData = DataLoader(datasetObject(encodedTestData,testTargets,testfingerprints),batch_size)
validationData = DataLoader(datasetObject(encodedValidationData,validationTargets,valfingerprints),batch_size)

In [9]:
from sklearn.metrics import r2_score
hidden_channels = 1024
embedding_dim = 128
d_target = 15
nHeads = 4
learningRate = 0.0005
nfingerprints = len(mol_fingerprint[0])
vocab_size = len(tokenDict)+1
torch.manual_seed(12345)

model = TransformerNetwork(hidden_channels,d_target,vocab_size, embedding_dim, nHeads,maxTokenLength,nfingerprints).to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=5e-4)
# decay learning rate
decayRate = 0.95
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
criterion = torch.nn.MSELoss().to(device)

def train(data_in, targets, fingerprints):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data_in,fingerprints).to(device)
      loss = criterion(out, targets)
      
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def infer(data_in, targets, fingerprints):
      with torch.no_grad():
            model.eval()
            out = model(data_in, fingerprints)
            loss = criterion(out, targets)
            r2 = r2_score(targets.cpu().detach().numpy(), out.cpu().detach().numpy())
      return loss, r2

# This calculates r2 for each targetr separately 
def inferNew(data_in, targets, fingerprints):
    with torch.no_grad():
        model.eval()
        out = model(data_in, fingerprints)
        loss = criterion(out, targets)

        r2_scores = []
        for item in range(targets.shape[1]):
            r2 = r2_score(targets[:, item].cpu().detach().numpy(), out[:, item].cpu().detach().numpy())
            r2_scores.append(r2)

    return loss, r2_scores





earlyStop = False
stopTolerance = 3
minDiff = 0.005
counter = 0
epoch = 0
lossList = []
r2List = []

while not earlyStop and epoch < 250:
    loss = 0
    for input, targets, descriptor in trainingData:
        tempLoss = train(input, targets, descriptor).detach()
        loss += tempLoss
    
    test = 0
    r2 = [0]*d_target
    for input, targets, descriptor in testData:
        tempTest, tempR2 = inferNew(input, targets, descriptor)
        test += tempTest
        # Add r2 for each variable
        r2 = [x + y for x, y in zip(r2, tempR2)]
    # Average over batch
    r2 = [x / len(testData) for x in r2]
    r2List.append(r2)
    

    val = 0
    for input, targets, descriptor in validationData:
        tempVal, tempR2 = infer(input, targets, descriptor)
        val += tempVal

    lr_scheduler.step()

    lossList.append([loss.item() / len(trainingData), test.item() / len(testData), val.item() / len(validationData)])

    # Early stopping
    if (lossList[epoch][2] - lossList[epoch][0]) > minDiff:
        counter += 1
    else:
        counter = 0
        if counter >= stopTolerance:
            earlyStop = True

    # Save loss as a Python number (not tensor) in the list
    print(f"#### Epoch: {epoch} of 1000 ####\n Training loss:\t{lossList[epoch][0]}\n Test loss:\t{lossList[epoch][1]}\n Validation loss:\t{lossList[epoch][2]}")
    print('R2:', r2List[epoch])
    print('avg R2',np.mean(r2List[epoch]),'\n')
    epoch += 1



#### Epoch: 0 of 1000 ####
 Training loss:	0.24784526790501768
 Test loss:	0.1427007271693303
 Validation loss:	0.14276660405672514
R2: [0.7155236866016257, 0.8029439239172879, 0.8438371272745109, 0.5904831584879697, 0.893049670410838, 0.7890060567949964, 0.9031452166700936, 0.8870017885957417, 0.8856089763866648, 0.924423604525668, 0.9041788598844411, 0.9040434971580198, 0.9046261040668383, 0.9041584259935967, 0.9055871526952198]
avg R2 0.8505078166309008 



KeyboardInterrupt: 